# 1. 사고발생이정 예측 모델 구축

## 1.1. 실습 개요

이 노트북에서는 고속도로 교통사고 데이터를 기반으로 **사고발생이정(사고 발생 지점의 기점 거리, km)** 을 예측하는 머신러닝 회귀 모델을 구축합니다.

## 1.2. 사고심각도 vs 사고발생이정 비교

| 항목 | 사고심각도 | 사고발생이정 |
|------|-----------|-------------|
| 정의 | 사망 × 3 + 부상 | 기점으로부터의 거리 (km) |
| 예측 가능성 | 낮음 (불규칙적) | 높음 (구조적 관계 존재) |
| 핵심 설명 변수 | 사망, 부상 (제외됨) | 노선명, 방향 |
| 예상 성능 | R² < 0 | R² > 0 (개선 기대) |

## 1.3. 학습 목표

1. 예측 가능한 타깃 변수의 특성 이해
2. 노선/방향 정보가 위치 예측에 미치는 영향 분석
3. Feature Importance를 통한 주요 변수 탐색
4. 사고심각도 예측 결과와 비교하여 타깃 선택의 중요성 체험

## 1.4. 사용 모델

이전 노트북과 동일한 4가지 회귀 모델을 사용합니다:
- Linear Regression (Baseline)
- RandomForest Regressor
- XGBoost Regressor
- LightGBM Regressor

In [ ]:
# ============================================
# 2. 패키지 설치 및 한글 폰트 설정
# ============================================

# --------------------------------------------
# 2.1. 추가 패키지 설치
# --------------------------------------------
!pip install xgboost lightgbm -q

# --------------------------------------------
# 2.2. 한글 폰트 설정 (코랩 전용)
# --------------------------------------------
!apt-get update -qq
!apt-get install -y fonts-nanum

import matplotlib as mpl
import shutil

root = mpl.matplotlib_fname().replace("matplotlibrc", "")
target_font = root + "fonts/ttf/DejaVuSans.ttf"
nanum_font = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
shutil.copyfile(nanum_font, target_font)

!rm -rf ~/.cache/matplotlib

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# 한글 출력 테스트
plt.figure(figsize=(4, 3))
plt.title("한글 폰트 정상 출력 확인")
plt.xlabel("가로축")
plt.ylabel("세로축")
plt.plot([1, 2, 3], [1, 4, 2])
plt.show()

In [ ]:
# ============================================
# 3. 라이브러리 임포트 및 데이터 로드
# ============================================

import numpy as np
import pandas as pd
from math import sqrt

# scikit-learn 관련 모듈
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 회귀 모델들
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 데이터 로드
file_path = "highway_accident_fe.csv"
df = pd.read_csv(file_path)

print("데이터 크기:", df.shape)
display(df.head())

In [ ]:
# ============================================
# 4. 타깃/특징 분리 및 피처 타입 분류
# ============================================

# --------------------------------------------
# 4.1. 타깃 변수 정의
# --------------------------------------------
# - 이번에는 사고발생이정을 예측 (사고심각도와 다른 타깃)
target_col = "사고발생이정"

# --------------------------------------------
# 4.2. 제외할 컬럼 정의
# --------------------------------------------
# - 사고발생이정: 예측 타깃이므로 X에서 제외
# - 사고심각도: 이번 타깃과 무관한 변수
# - 사고일자: 고유값이 너무 많음
drop_cols = ["사고발생이정", "사고심각도", "사고일자"]
drop_cols = [c for c in drop_cols if c in df.columns]

# X: 입력 특징, y: 타깃
X = df.drop(columns=drop_cols)
y = df[target_col]

print("입력 특징 컬럼:", X.columns.tolist())

# --------------------------------------------
# 4.3. 범주형 / 수치형 컬럼 자동 분류
# --------------------------------------------
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numeric_cols = X.select_dtypes(include=["int64", "float64", "int32", "float32"]).columns.tolist()

print("\n범주형 컬럼:", categorical_cols)
print("수치형 컬럼:", numeric_cols)

In [ ]:
# ============================================
# 5. 전처리기(Preprocessor) 구성
# ============================================

# - 범주형: OneHotEncoder
# - 수치형: passthrough (그대로 통과)
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

In [ ]:
# ============================================
# 6. 학습/테스트 데이터 분리
# ============================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("학습 데이터 크기:", X_train.shape)
print("테스트 데이터 크기:", X_test.shape)

In [ ]:
# ============================================
# 7. 회귀 모델 정의
# ============================================

models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1,
    ),
    "XGBoost": XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=300,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    "LinearRegression": LinearRegression()
}

# 성능 저장용 리스트
metrics_list = []

# Feature Importance 저장용 딕셔너리
feature_importances_dict = {}

In [ ]:
# ============================================
# 8. 모델 학습 및 평가
# ============================================

for model_name, model in models.items():
    print(f"\n{'=' * 50}")
    print(f"{model_name} 학습 중...")
    print("=" * 50)
    
    # --------------------------------------------
    # 8.1. Pipeline 구성 및 학습
    # --------------------------------------------
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model),
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    # --------------------------------------------
    # 8.2. 평가 지표 계산
    # --------------------------------------------
    mae = mean_absolute_error(y_test, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    metrics_list.append({
        "model": model_name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    })
    
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    # --------------------------------------------
    # 8.3. Feature Importance 추출
    # --------------------------------------------
    preproc = pipe.named_steps["preprocessor"]
    feature_names = preproc.get_feature_names_out()
    fitted_model = pipe.named_steps["model"]
    
    if hasattr(fitted_model, "feature_importances_"):
        importances = fitted_model.feature_importances_
    elif hasattr(fitted_model, "coef_"):
        importances = np.abs(fitted_model.coef_)
    else:
        importances = None
    
    if importances is not None:
        fi_df = pd.DataFrame({
            "feature": feature_names,
            "importance": importances
        }).sort_values("importance", ascending=False)
        
        feature_importances_dict[model_name] = fi_df
        print(f"\n[{model_name}] 상위 10개 Feature Importance:")
        print(fi_df.head(10).to_string(index=False))

In [ ]:
# ============================================
# 9. 모델 성능 비교 및 시각화
# ============================================

metrics_df = pd.DataFrame(metrics_list)
print("\n" + "=" * 50)
print("모델 성능 비교 (타깃: 사고발생이정)")
print("=" * 50)
display(metrics_df)

# RMSE 기준 정렬
metrics_df_sorted = metrics_df.sort_values("RMSE")
print("\n[RMSE 기준 정렬]")
display(metrics_df_sorted)

# --------------------------------------------
# 9.1. RMSE 막대그래프
# --------------------------------------------
plt.figure(figsize=(8, 4))
sns.barplot(data=metrics_df_sorted, x="model", y="RMSE", palette="Blues_d")
plt.title("모델별 RMSE 비교 (타깃: 사고발생이정)")
plt.xlabel("모델")
plt.ylabel("RMSE")
plt.tight_layout()
plt.show()

# --------------------------------------------
# 9.2. R² 막대그래프
# --------------------------------------------
plt.figure(figsize=(8, 4))
sns.barplot(
    data=metrics_df.sort_values("R2", ascending=False),
    x="model", y="R2", palette="Greens_d"
)
plt.title("모델별 R² 비교 (타깃: 사고발생이정)")
plt.xlabel("모델")
plt.ylabel("R²")
plt.axhline(y=0, color="gray", linestyle="--", linewidth=1)
plt.tight_layout()
plt.show()

# 10. 결과 해석 및 실습 정리

## 10.1. 핵심 인사이트: Feature Importance 분석

모든 모델에서 공통적으로 **노선명**이 가장 중요한 변수로 나타났습니다.

| 모델 | 상위 중요 피처 |
|------|----------------|
| RandomForest | 경부선, 사고분, 사고일, 서해안선, 사고시 |
| XGBoost | 경부선, 방향_하남, 중앙선, 제2중부선 |
| LightGBM | 사고분, 사고일, 사고시, 사고월 |
| LinearRegression | 수도권제2순환선, 경부선, 제2중부선 |

**해석**: 노선명은 사고발생이정(km)과 직접적인 관계가 있습니다.
- 각 노선마다 고유한 길이와 특성이 있음
- 경부선은 길이가 길어 사고발생이정 값의 범위가 넓음
- 방향 정보도 위치 예측에 도움이 됨

## 10.2. 사고심각도 vs 사고발생이정 비교

| 항목 | 사고심각도 예측 | 사고발생이정 예측 |
|------|----------------|------------------|
| R² 결과 | 음수 (baseline보다 나쁨) | 양수 또는 개선됨 |
| 핵심 변수 | 사망/부상 (제외됨) | 노선명/방향 (포함됨) |
| 예측 난이도 | 매우 어려움 | 상대적으로 쉬움 |

## 10.3. 3일간 실습 정리

### Day 1: 데이터 수집 및 기초 분석
- API를 활용한 공공데이터 수집
- 기초 시각화 및 통계량 확인

### Day 2: EDA 심화 및 모델링
- Feature Engineering (파생변수 생성)
- 다차원 EDA (노선별, 시간대별, 원인별 분석)
- 사고심각도 예측 모델 구축 및 한계 확인

### Day 3: 타깃 재설정 및 모델링
- 사고발생이정 예측 모델 구축
- 타깃 선택의 중요성 체험
- Feature Importance를 통한 인사이트 도출

## 10.4. 핵심 교훈

1. **타깃 변수 선택이 중요**: 예측하려는 것과 관련된 정보가 데이터에 있어야 함
2. **데이터 누설(Leakage) 주의**: 타깃과 직접 관련된 변수는 제외해야 함
3. **도메인 지식 활용**: 노선명이 위치와 관련있다는 도메인 지식이 모델 해석에 도움
4. **모델은 도구일 뿐**: 좋은 모델도 좋은 데이터 없이는 무용지물